<a href="https://colab.research.google.com/github/robb2828/fournier-coding/blob/main/Fall_2021_Intern_Data_Science_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fall 2021 Intern Data Science Challenge

Prepared by: Robb Fournier

For: Shopify